In [ ]:
# MICE 구현을 위한 라이브러리 설치
!pip install fancyimpute openpyxl

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 9.3 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=e701b767ba6c022a623c9996f6a8886ffcf9d2e18d273b4dc3ca22d1490bc690
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=9d24eb4fe399895aa42462bd5ddeb383935e102169c685b885c23c3420491a63
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
import pandas as pd
from google.colab import files

# 파일 업로드
uploaded = files.upload()  # 파일 업로드 창이 뜹니다

Saving 온열질환환자데이터.xlsx to 온열질환환자데이터.xlsx


In [ ]:
# 업로드된 파일의 이름을 가져옵니다
file_path = list(uploaded.keys())[0]

# 데이터 로드 (엑셀 파일)
data = pd.read_excel(file_path)

In [ ]:
from google.colab import data_table
# AG열(CRP) 값에서 화살표 제거
data['CRP'] = data['CRP'].astype(str)  # 문자열로 변환
data['CRP'] = data['CRP'].str.replace('↓', '', regex=False)  # 화살표 제거

# 데이터 타입 변환 (숫자로 변환)
data['CRP'] = pd.to_numeric(data['CRP'], errors='coerce')  # 변환 불가능한 값은 NaN 처리

# 결과 확인
print(data['CRP'].head())

subset_data = data.iloc[:, 20:33]  # 열 범위 확인

0    17.400
1     0.460
2     0.160
3     1.000
4     0.103
Name: CRP, dtype: float64


In [ ]:
from fancyimpute import IterativeImputer

# MICE 대체
mice_imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = mice_imputer.fit_transform(subset_data)

# 결과를 DataFrame으로 변환
imputed_df = pd.DataFrame(imputed_data, columns=subset_data.columns)


In [ ]:
# 결측치 확인
print("결측치 개수 (Before):")
print(subset_data.isnull().sum())

print("\n결측치 개수 (After):")
print(pd.DataFrame(imputed_data, columns=subset_data.columns).isnull().sum())

# 원본 데이터에 대체된 데이터 반영
data.iloc[:, 20:33] = imputed_df


결측치 개수 (Before):
GCS     95
SBP    233
BT     475
HR     341
WBC    325
Ht     388
Plt    378
BUN    353
Cre    338
AST    370
ALT    374
CK     555
CRP    536
dtype: int64

결측치 개수 (After):
GCS    0
SBP    0
BT     0
HR     0
WBC    0
Ht     0
Plt    0
BUN    0
Cre    0
AST    0
ALT    0
CK     0
CRP    0
dtype: int64


In [ ]:
# 엑셀 파일로 저장
output_path = '/content/modified_data.xlsx'  # 저장할 파일 경로
data.to_excel(output_path, index=False)  # index=False는 인덱스를 제외하고 저장

# 파일 다운로드
from google.colab import files
#files.download(output_path)  # 파일 다운로드


In [ ]:
# 열 제거 (N열 PreSBP부터 R열 PreHR까지)
columns_to_drop = data.loc[:, "PreSBP":"PreHR"].columns  # 제거할 열 이름 확인
data = data.drop(columns=columns_to_drop)

# 결과 확인
print("제거 후 데이터프레임:")
print(data.head())

제거 후 데이터프레임:
   Sex_1male_2female   Age  Weather_1sunny_2cloudy_3rainy_4missing  \
0                1.0  24.0                                     3.0   
1                NaN  43.0                                     NaN   
2                1.0  58.0                                     NaN   
3                1.0  46.0                                     1.0   
4                1.0  57.0                                     1.0   

   Location_1outdoor_indoor  Unnamed: 4  HT  HeartDisease  Pscyco  DM  CerevD  \
0                       NaN         1.0   0             0       0   0       0   
1                       1.0         NaN   0             0       0   0       0   
2                       1.0         1.0   0             0       0   0       0   
3                       1.0         1.0   0             0       0   0       0   
4                       1.0         1.0   0             0       0   0       0   

   ...   Plt   BUN   Cre   AST   ALT     CK     CRP  Admission1  ICU  \
0  ... 

In [ ]:
#A열 부터
# 분석할 열 선택 (A열부터 T열까지)
columns_to_analyze = data.loc[:, "Sex_1male_2female":"Muscular"]

# 각 열의 결측치 비율 계산
missing_ratio = columns_to_analyze.isnull().sum() / len(columns_to_analyze)

# 결과 출력
print("결측치 비율:")
print(missing_ratio)

# 비율이 높은 열만 필터링 (예: 10% 이상)
high_missing_columns = missing_ratio[missing_ratio > 0.1]
print("\n결측치 비율이 높은 열:")
print(high_missing_columns)

결측치 비율:
Sex_1male_2female                         0.020472
Age                                       0.012598
Weather_1sunny_2cloudy_3rainy_4missing    0.191496
Location_1outdoor_indoor                  0.423307
Unnamed: 4                                0.119055
HT                                        0.000000
HeartDisease                              0.000000
Pscyco                                    0.000000
DM                                        0.000000
CerevD                                    0.000000
ParkinD                                   0.000000
CKD                                       0.000000
Dementia                                  0.000000
Abdminal                                  0.427087
Muscular                                  0.504567
dtype: float64

결측치 비율이 높은 열:
Weather_1sunny_2cloudy_3rainy_4missing    0.191496
Location_1outdoor_indoor                  0.423307
Unnamed: 4                                0.119055
Abdminal                                  0.

In [ ]:
# KNN 방법을 통한 결측치 처리
!pip install scikit-learn

# 라이브러리 불러오기
from sklearn.impute import KNNImputer

In [ ]:
# 필요한 열만 서브셋으로 선택
columns_to_impute = ["Sex_1male_2female", "Age", "Weather_1sunny_2cloudy_3rainy_4missing"]
subset_data = data[columns_to_impute]

# 선택한 데이터 확인
print(subset_data.head())


   Sex_1male_2female   Age  Weather_1sunny_2cloudy_3rainy_4missing
0                1.0  24.0                                     3.0
1                NaN  43.0                                     NaN
2                1.0  58.0                                     NaN
3                1.0  46.0                                     1.0
4                1.0  57.0                                     1.0


In [ ]:
# KNN Imputer 초기화
knn_imputer = KNNImputer(n_neighbors=3)  # 이웃 3개 사용

In [ ]:
# KNN을 적용하여 결측치 대체
imputed_data = knn_imputer.fit_transform(subset_data)

# DataFrame으로 변환
imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute)

# 결과 확인
print(imputed_df.head())


   Sex_1male_2female   Age  Weather_1sunny_2cloudy_3rainy_4missing
0                1.0  24.0                                     3.0
1                1.0  43.0                                     1.0
2                1.0  58.0                                     1.0
3                1.0  46.0                                     1.0
4                1.0  57.0                                     1.0


In [ ]:
# 원본 데이터 업데이트
data.update(imputed_df)

# 결측치 확인
print("결측치 처리 후:")
print(data[columns_to_impute].isnull().sum())

결측치 처리 후:
Sex_1male_2female                         0
Age                                       0
Weather_1sunny_2cloudy_3rainy_4missing    0
dtype: int64


In [ ]:
# 데이터 저장 경로와 파일 이름 설정
file_name = "processed_data.xlsx"

# 데이터프레임을 엑셀로 저장
data.to_excel(file_name, index=False)

# 파일 다운로드 링크 제공
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>